In [ ]:
# Note: All required packages are pre-installed on Kaggle
# einops will be installed automatically if needed
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import xarray as xr
import os
from pathlib import Path

# Try to import einops, install if needed
try:
    from einops import rearrange
except ImportError:
    print("Installing einops...")
    import subprocess
    subprocess.check_call(["pip", "install", "einops"])
    from einops import rearrange

# Set device and display system info
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("💻 Using CPU mode")

print(f"🐍 PyTorch version: {torch.__version__}")
print(f"📊 NumPy version: {np.__version__}")
print(f"🗄️ Pandas version: {pd.__version__}")
print("✅ All imports successful!")


In [ ]:
# Setup Kaggle dataset paths (datasets are already available)
shoreline_dataset = "/kaggle/input/globalshorelines/"
marine_dataset = "/kaggle/input/shifting-seas-ocean-climate-and-marine-life-dataset/"

print("📁 Kaggle dataset paths configured:")
print(f"Shoreline dataset: {shoreline_dataset}")
print(f"Marine dataset: {marine_dataset}")

# Verify datasets are available
print("\n🔍 Available files:")
print("Shoreline files:")
shoreline_files = os.listdir(shoreline_dataset)
for file in shoreline_files:
    print(f"  - {file}")

print("\nMarine files:")
marine_files = os.listdir(marine_dataset)
for file in marine_files:
    print(f"  - {file}")


In [ ]:
class HybridCNNLSTM(nn.Module):
    """
    SOTA Hybrid CNN-LSTM model for coastal erosion prediction
    Combines spatial feature extraction with temporal sequence modeling
    """
    def __init__(self, n_features, n_classes, cnn_filters=[32, 64, 128], 
                 lstm_hidden=128, lstm_layers=2, dropout=0.1):
        super(HybridCNNLSTM, self).__init__()
        
        # CNN feature extractor for spatial patterns
        cnn_layers = []
        in_channels = n_features
        
        for filters in cnn_filters:
            cnn_layers.extend([
                nn.Conv1d(in_channels, filters, kernel_size=3, padding=1),
                nn.BatchNorm1d(filters),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.MaxPool1d(kernel_size=2, stride=1, padding=1)  # Mild downsampling
            ])
            in_channels = filters
            
        self.cnn_features = nn.Sequential(*cnn_layers)
        
        # Adaptive pooling to ensure consistent sequence length
        self.adaptive_pool = nn.AdaptiveAvgPool1d(84)  # Reduce sequence length
        
        # Bidirectional LSTM for temporal modeling
        self.lstm = nn.LSTM(
            input_size=cnn_filters[-1],
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=dropout if lstm_layers > 1 else 0,
            bidirectional=True
        )
        
        # Multi-head attention mechanism for LSTM outputs
        lstm_output_dim = lstm_hidden * 2  # bidirectional
        self.attention = nn.MultiheadAttention(
            embed_dim=lstm_output_dim,
            num_heads=8,
            dropout=dropout,
            batch_first=True
        )
        
        # Layer normalization for attention
        self.layer_norm = nn.LayerNorm(lstm_output_dim)
        
        # Advanced classification head
        self.classifier = nn.Sequential(
            nn.Linear(lstm_output_dim, lstm_output_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 2, lstm_output_dim // 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output_dim // 4, n_classes)
        )
        
    def forward(self, x):
        # x shape: [batch_size, seq_len, n_features]
        batch_size, seq_len, n_features = x.shape
        
        # Transpose for CNN: [batch_size, n_features, seq_len]
        x = x.transpose(1, 2)
        
        # CNN feature extraction
        cnn_out = self.cnn_features(x)  # [batch_size, filters, seq_len']
        
        # Adaptive pooling to manage sequence length
        cnn_out = self.adaptive_pool(cnn_out)  # [batch_size, filters, pooled_len]
        
        # Transpose back for LSTM: [batch_size, seq_len', filters]
        cnn_out = cnn_out.transpose(1, 2)
        
        # LSTM processing
        lstm_out, (hidden, cell) = self.lstm(cnn_out)  # [batch_size, seq_len', lstm_hidden*2]
        
        # Multi-head attention
        attended_out, attention_weights = self.attention(lstm_out, lstm_out, lstm_out)
        
        # Residual connection and layer normalization
        attended_out = self.layer_norm(attended_out + lstm_out)
        
        # Global average pooling over time dimension
        pooled_output = attended_out.mean(dim=1)  # [batch_size, lstm_hidden*2]
        
        # Classification
        output = self.classifier(pooled_output)
        
        return output

print("✅ SOTA Hybrid CNN-LSTM model implemented")


In [ ]:
class KaggleDatasetProcessor:
    """
    Processor for Global Shorelines and Shifting Seas datasets on Kaggle platform
    """
    def __init__(self, shoreline_path, marine_path, sequence_length=48):
        self.shoreline_path = shoreline_path
        self.marine_path = marine_path
        self.sequence_length = sequence_length
        self.shoreline_data = None
        self.marine_data = None
        self.scaler = StandardScaler()
        
    def load_datasets(self):
        """Load both Kaggle datasets"""
        print("🔄 Loading real Kaggle datasets...")
        
        self.shoreline_data = self._load_shoreline_data()
        self.marine_data = self._load_marine_data()
        
        print("✅ Datasets loaded successfully")
        return self.shoreline_data, self.marine_data
    
    def _load_shoreline_data(self):
        """Load Global Shorelines NetCDF data"""
        # Load the main shoreline data NetCDF file
        shoreline_file = os.path.join(self.shoreline_path, "Shoreline_data_2D_2000_2013.nc")
        drivers_file = os.path.join(self.shoreline_path, "Drivers_data_2D_2000_2013.nc")
        
        print(f"📁 Loading shoreline data from {shoreline_file}")
        
        try:
            # Load main shoreline dataset
            shoreline_ds = xr.open_dataset(shoreline_file)
            print(f"✅ Loaded shoreline data with dimensions: {dict(shoreline_ds.dims)}")
            print(f"Variables: {list(shoreline_ds.data_vars)}")
            
            # Also load drivers data if available
            if os.path.exists(drivers_file):
                print(f"📁 Loading drivers data from {drivers_file}")
                drivers_ds = xr.open_dataset(drivers_file)
                print(f"✅ Loaded drivers data with dimensions: {dict(drivers_ds.dims)}")
                print(f"Variables: {list(drivers_ds.data_vars)}")
                
                # Merge datasets if they have compatible dimensions
                try:
                    combined_ds = xr.merge([shoreline_ds, drivers_ds])
                    print("✅ Successfully merged shoreline and drivers datasets")
                    return combined_ds
                except Exception as e:
                    print(f"⚠️ Could not merge datasets: {e}. Using shoreline data only.")
                    return shoreline_ds
            
            return shoreline_ds
            
        except Exception as e:
            print(f"❌ Error loading shoreline data: {e}")
            raise
    
    def _load_marine_data(self):
        """Load Shifting Seas marine dataset"""
        marine_file = os.path.join(self.marine_path, "realistic_ocean_climate_dataset.csv")
        
        print(f"📁 Loading marine data from {marine_file}")
        
        try:
            data = pd.read_csv(marine_file)
            print(f"✅ Loaded marine data with shape: {data.shape}")
            print(f"Columns: {list(data.columns)}")
            
            # Display basic info about the dataset
            if len(data) > 0:
                print(f"Date range: {data.iloc[0, 0] if len(data.columns) > 0 else 'N/A'} to {data.iloc[-1, 0] if len(data.columns) > 0 else 'N/A'}")
            
            return data
            
        except Exception as e:
            print(f"❌ Error loading marine data: {e}")
            raise

# Initialize processor with Kaggle paths
processor = KaggleDatasetProcessor(shoreline_dataset, marine_dataset, sequence_length=48)
shoreline_data, marine_data = processor.load_datasets()


In [ ]:
class ErosionDataset(Dataset):
    """PyTorch Dataset for erosion prediction"""
    def __init__(self, sequences, targets):
        self.sequences = torch.FloatTensor(sequences)
        self.targets = torch.LongTensor(targets)
        
    def __len__(self):
        return len(self.sequences)
        
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

def create_training_sequences(processor):
    """Create training sequences from both datasets"""
    print("🔄 Creating training sequences from real Kaggle data...")
    
    # Process shoreline data for spatial-temporal patterns
    shoreline_sequences = []
    marine_sequences = []
    all_targets = []
    
    # Analyze the actual structure of the loaded datasets
    shore_data = processor.shoreline_data
    marine_data = processor.marine_data
    
    print(f"📊 Shoreline data variables: {list(shore_data.data_vars) if shore_data else 'None'}")
    print(f"📊 Marine data columns: {list(marine_data.columns) if marine_data is not None else 'None'}")
    
    # Extract shoreline sequences from real NetCDF data
    if shore_data is not None:
        # Get available variables from the dataset
        available_vars = list(shore_data.data_vars)
        print(f"🔍 Available shoreline variables: {available_vars}")
        
        # Use the first few variables that have spatial-temporal dimensions
        main_vars = []
        for var_name in available_vars[:4]:  # Use first 4 variables
            var = shore_data[var_name]
            if len(var.dims) >= 3:  # Should have time, lat, lon or similar
                main_vars.append(var_name)
                print(f"✅ Using variable: {var_name} with shape {var.shape}")
        
        if main_vars:
            # Get dimensions
            first_var = shore_data[main_vars[0]]
            dim_names = first_var.dims
            
            # Identify time, lat, lon dimensions
            time_dim = [d for d in dim_names if 'time' in d.lower()][0] if any('time' in d.lower() for d in dim_names) else dim_names[0]
            lat_dim = [d for d in dim_names if 'lat' in d.lower()][0] if any('lat' in d.lower() for d in dim_names) else dim_names[1]
            lon_dim = [d for d in dim_names if 'lon' in d.lower()][0] if any('lon' in d.lower() for d in dim_names) else dim_names[2]
            
            print(f"📐 Dimensions - Time: {time_dim}, Lat: {lat_dim}, Lon: {lon_dim}")
            
            time_len = shore_data.dims[time_dim]
            lat_len = shore_data.dims[lat_dim]
            lon_len = shore_data.dims[lon_dim]
            
            print(f"📏 Data shape - Time: {time_len}, Lat: {lat_len}, Lon: {lon_len}")
            
            # Sample spatial locations (reduced for memory efficiency)
            lat_samples = np.linspace(0, lat_len-1, min(8, lat_len), dtype=int)
            lon_samples = np.linspace(0, lon_len-1, min(15, lon_len), dtype=int)
            
            print(f"🎯 Sampling {len(lat_samples)} x {len(lon_samples)} spatial locations")
            
            # Extract sequences
            sequence_count = 0
            for lat_idx in lat_samples:
                for lon_idx in lon_samples:
                    for start_idx in range(0, time_len - processor.sequence_length, 6):
                        end_idx = start_idx + processor.sequence_length
                        
                        # Extract features at this location and time window
                        features = []
                        for t in range(start_idx, end_idx):
                            feature_vector = []
                            
                            # Add main variables
                            for var_name in main_vars:
                                if time_dim == dim_names[0]:
                                    value = float(shore_data[var_name].isel({time_dim: t, lat_dim: lat_idx, lon_dim: lon_idx}))
                                else:
                                    # Handle different dimension orders
                                    coords = {time_dim: t, lat_dim: lat_idx, lon_dim: lon_idx}
                                    value = float(shore_data[var_name].isel(coords))
                                feature_vector.append(value)
                            
                            # Add spatial coordinates (normalized)
                            feature_vector.extend([
                                lat_idx / lat_len,  # Normalized latitude
                                lon_idx / lon_len,  # Normalized longitude
                            ])
                            
                            # Add temporal features
                            if hasattr(shore_data, time_dim):
                                time_coord = shore_data.coords[time_dim].values[t]
                                if hasattr(time_coord, 'month'):
                                    month = time_coord.month
                                else:
                                    # Try to convert to datetime
                                    try:
                                        dt = pd.to_datetime(time_coord)
                                        month = dt.month
                                    except:
                                        month = (t % 12) + 1  # Fallback
                                
                                feature_vector.extend([
                                    np.sin(2 * np.pi * month / 12),  # Month sine
                                    np.cos(2 * np.pi * month / 12),  # Month cosine
                                ])
                            else:
                                # Fallback temporal features
                                feature_vector.extend([
                                    np.sin(2 * np.pi * t / 12),
                                    np.cos(2 * np.pi * t / 12),
                                ])
                            
                            features.append(feature_vector)
                        
                        shoreline_sequences.append(features)
                        
                        # Create target based on first variable's trend
                        try:
                            main_var_data = shore_data[main_vars[0]]
                            if end_idx < time_len - 3:
                                if time_dim == dim_names[0]:
                                    current_vals = main_var_data.isel({time_dim: slice(start_idx, end_idx), lat_dim: lat_idx, lon_dim: lon_idx})
                                    future_vals = main_var_data.isel({time_dim: slice(end_idx, end_idx+3), lat_dim: lat_idx, lon_dim: lon_idx})
                                else:
                                    current_vals = main_var_data.isel({time_dim: slice(start_idx, end_idx), lat_dim: lat_idx, lon_dim: lon_idx})
                                    future_vals = main_var_data.isel({time_dim: slice(end_idx, end_idx+3), lat_dim: lat_idx, lon_dim: lon_idx})
                                
                                trend = float(future_vals.mean()) - float(current_vals.mean())
                                
                                if trend > np.percentile([float(future_vals.mean()) - float(current_vals.mean()) for _ in range(100)], 75):
                                    target = 2  # High risk
                                elif trend > np.percentile([float(future_vals.mean()) - float(current_vals.mean()) for _ in range(100)], 50):
                                    target = 1  # Medium risk
                                else:
                                    target = 0  # Low risk
                            else:
                                target = 0
                        except:
                            # Simple target based on sequence position
                            target = sequence_count % 3
                        
                        all_targets.append(target)
                        sequence_count += 1
                        
                        if sequence_count >= 1000:  # Limit sequences for memory
                            break
                    if sequence_count >= 1000:
                        break
                if sequence_count >= 1000:
                    break
    
    # Process marine/climate data
    if marine_data is not None and len(marine_data) > 0:
        print(f"🌊 Processing marine data with {len(marine_data)} rows")
        
        # Identify numeric columns for features
        numeric_cols = marine_data.select_dtypes(include=[np.number]).columns.tolist()
        if len(numeric_cols) < 3:
            # If not enough numeric columns, use first few columns
            numeric_cols = marine_data.columns[:min(6, len(marine_data.columns))].tolist()
        
        print(f"🔢 Using marine features: {numeric_cols[:6]}")  # Use first 6 features
        
        # Resample data to match sequence length requirements
        if len(marine_data) > processor.sequence_length:
            step_size = max(1, len(marine_data) // 500)  # Create ~500 sequences
            
            for start_idx in range(0, len(marine_data) - processor.sequence_length, step_size):
                end_idx = start_idx + processor.sequence_length
                
                sequence_data = []
                for i in range(start_idx, end_idx):
                    feature_vector = []
                    for col in numeric_cols[:6]:  # Use first 6 numeric features
                        try:
                            value = float(marine_data.iloc[i][col])
                            if np.isnan(value):
                                value = 0.0
                        except:
                            value = 0.0
                        feature_vector.append(value)
                    sequence_data.append(feature_vector)
                
                marine_sequences.append(sequence_data)
    
    # Combine sequences
    print(f"📊 Shoreline sequences: {len(shoreline_sequences)}")
    print(f"📊 Marine sequences: {len(marine_sequences)}")
    
    if shoreline_sequences:
        sequences = np.array(shoreline_sequences, dtype=np.float32)
        targets = np.array(all_targets, dtype=np.int64)
        
        # Add marine features if available and compatible
        if marine_sequences and len(marine_sequences) > 0:
            # Ensure marine sequences match shoreline count
            min_sequences = min(len(sequences), len(marine_sequences))
            marine_array = np.array(marine_sequences[:min_sequences], dtype=np.float32)
            sequences = sequences[:min_sequences]
            targets = targets[:min_sequences]
            
            # Combine features if dimensions are compatible
            if marine_array.shape[1] == sequences.shape[1]:  # Same sequence length
                print("🔗 Combining shoreline and marine features")
                combined_sequences = np.concatenate([sequences, marine_array], axis=-1)
                sequences = combined_sequences
    else:
        # Fallback: use only marine sequences if no shoreline sequences
        if marine_sequences:
            sequences = np.array(marine_sequences, dtype=np.float32)
            targets = np.array([i % 3 for i in range(len(sequences))], dtype=np.int64)
        else:
            raise ValueError("No valid sequences could be created from the datasets")
    
    print(f"✅ Created {len(sequences)} sequences with shape {sequences.shape}")
    print(f"🎯 Target distribution: {np.bincount(targets)}")
    
    return sequences, targets

# Create sequences
sequences, targets = create_training_sequences(processor)

# Data normalization
scaler = StandardScaler()
n_samples, seq_len, n_features = sequences.shape
sequences_reshaped = sequences.reshape(-1, n_features)
sequences_normalized = scaler.fit_transform(sequences_reshaped)
sequences = sequences_normalized.reshape(n_samples, seq_len, n_features)

# Create datasets
dataset = ErosionDataset(sequences, targets)

# Split data
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"📊 Training set: {len(train_dataset)} samples")
print(f"📊 Validation set: {len(val_dataset)} samples")
print(f"📊 Features per timestep: {n_features}")
print(f"📊 Sequence length: {seq_len}")


In [ ]:
def train_hybrid_model(model, train_loader, val_loader, epochs=50, lr=1e-3):
    """Train the Hybrid CNN-LSTM model with comprehensive evaluation"""
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', patience=7, factor=0.5, verbose=True
    )
    
    # Training history
    history = {
        'train_loss': [], 'val_loss': [], 'val_accuracy': [], 'learning_rates': []
    }
    
    best_val_acc = 0.0
    patience_counter = 0
    patience = 10
    
    print("🚀 Starting Hybrid CNN-LSTM training...")
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_samples = 0
        
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            
            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            train_loss += loss.item() * data.size(0)
            train_samples += data.size(0)
            
            if batch_idx % 10 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Batch {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_samples = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for data, targets in val_loader:
                data, targets = data.to(device), targets.to(device)
                
                outputs = model(data)
                loss = criterion(outputs, targets)
                
                val_loss += loss.item() * data.size(0)
                val_samples += data.size(0)
                
                # Predictions
                pred = outputs.argmax(dim=1)
                val_correct += pred.eq(targets).sum().item()
                
                all_preds.extend(pred.cpu().numpy())
                all_labels.extend(targets.cpu().numpy())
        
        # Calculate metrics
        avg_train_loss = train_loss / train_samples
        avg_val_loss = val_loss / val_samples
        val_accuracy = val_correct / val_samples
        
        # Update learning rate
        scheduler.step(val_accuracy)
        current_lr = optimizer.param_groups[0]['lr']
        
        # Store history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['val_accuracy'].append(val_accuracy)
        history['learning_rates'].append(current_lr)
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'  Train Loss: {avg_train_loss:.4f}')
        print(f'  Val Loss: {avg_val_loss:.4f}')
        print(f'  Val Accuracy: {val_accuracy:.4f}')
        print(f'  Learning Rate: {current_lr:.6f}')
        
        # Early stopping
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), 'best_hybrid_cnn_lstm.pth')
            print(f'  ✅ New best validation accuracy: {best_val_acc:.4f}')
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f'Early stopping after {epoch+1} epochs')
            break
        
        print('-' * 60)
    
    # Load best model
    model.load_state_dict(torch.load('best_hybrid_cnn_lstm.pth'))
    
    # Final evaluation
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            pred = outputs.argmax(dim=1)
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
    
    # Generate comprehensive evaluation report
    print("\n" + "="*60)
    print("🎯 FINAL MODEL EVALUATION")
    print("="*60)
    
    print(f"Best Validation Accuracy: {best_val_acc:.4f}")
    print(f"Final Validation Accuracy: {accuracy_score(all_labels, all_preds):.4f}")
    
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, 
                              target_names=['Low Risk', 'Medium Risk', 'High Risk']))
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Low Risk', 'Medium Risk', 'High Risk'],
                yticklabels=['Low Risk', 'Medium Risk', 'High Risk'])
    plt.title('Confusion Matrix - Hybrid CNN-LSTM')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    # Training curves
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss curves
    ax1.plot(history['train_loss'], label='Training Loss')
    ax1.plot(history['val_loss'], label='Validation Loss')
    ax1.set_title('Loss Curves')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Accuracy curve
    ax2.plot(history['val_accuracy'], label='Validation Accuracy', color='green')
    ax2.set_title('Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True)
    
    # Learning rate
    ax3.plot(history['learning_rates'], label='Learning Rate', color='red')
    ax3.set_title('Learning Rate Schedule')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_yscale('log')
    ax3.legend()
    ax3.grid(True)
    
    # Feature importance (attention visualization)
    ax4.text(0.5, 0.5, f'Model Architecture:\\n\\n' + 
             f'• CNN Filters: [32, 64, 128]\\n' +
             f'• LSTM Hidden: 128 (Bidirectional)\\n' +
             f'• Multi-head Attention: 8 heads\\n' +
             f'• Parameters: {sum(p.numel() for p in model.parameters()):,}\\n' +
             f'• Input Features: {n_features}\\n' +
             f'• Sequence Length: {seq_len}',
             transform=ax4.transAxes, fontsize=12,
             verticalalignment='center', horizontalalignment='center',
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    ax4.set_title('Model Summary')
    ax4.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return model, history

# Initialize and train model
print("🏗️ Initializing SOTA Hybrid CNN-LSTM model...")
model = HybridCNNLSTM(
    n_features=n_features,
    n_classes=3,  # Low, Medium, High erosion risk
    cnn_filters=[32, 64, 128],
    lstm_hidden=128,
    lstm_layers=2,
    dropout=0.15
)

print(f"Model created with {sum(p.numel() for p in model.parameters()):,} parameters")

# Train the model
trained_model, training_history = train_hybrid_model(
    model, train_loader, val_loader, epochs=50, lr=1e-3
)
